In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import re

# 데이터 로드
df = pd.read_csv('../data/Clean_Red_data.csv')

# 최소 리뷰 수와 평균 평점
m = df['wine_reviews'].min()
C = df['wine_rating'].mean()

def weighted_rating(x, m=m, C=C):
    v = x['wine_reviews']
    R = x['wine_rating']
    return (v/(v+m) * R) + (m/(v+m) * C)

df['point'] = df.apply(weighted_rating, axis=1)

# 주요 특성만 선택
df_features = df[['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']].copy()

# 'wine_price' 문자열에서 쉼표 제거 및 숫자형으로 변환
df['wine_price'] = df['wine_price'].str.replace(',', '').astype(float)

# 정규화 (0에서 1 사이로 변환)
for column in ['body', 'texture', 'sweetness', 'acidity']:
    df_features[column] = df_features[column] / 100.0

# ColumnTransformer 정의: 원-핫 인코딩 및 표준화 적용
ct = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(drop='first'), ['flavor1', 'flavor2', 'flavor3']),  # 범주형 데이터 원-핫 인코딩
        ('scaler', StandardScaler(), ['body', 'texture', 'sweetness', 'acidity'])  # 숫자형 데이터 표준화
    ],
    remainder='passthrough'
)

# 변환 적용
X = ct.fit_transform(df_features)

def calculate_distance_with_flavors(df, target_index, ct, columns):
    # 모든 feature를 인코딩 및 스케일링된 형태로 변환
    df_encoded = ct.transform(df[columns])
    
    # 특정 와인 벡터
    target_vector = df_encoded[target_index, :].reshape(1, -1)
    
    # 데이터 벡터
    data_vectors = df_encoded
    
    # 유클리디안 거리 계산
    distance_scores = euclidean_distances(target_vector, data_vectors)
    
    # 결과 반환 (거리 값이므로, 오름차순으로 정렬을 위해 음수 처리)
    return distance_scores.flatten()

# 와인 이름으로 인덱스 찾기
def find_wine_index(df, wine_name):
    return df[df['wine_name'] == wine_name].index[0]

def normalize_wine_name(name):
    """ 와인 이름에서 숫자와 공백을 제외하고 기본 이름만 반환 """
    return re.sub(r'\s+\d+$', '', name).strip()

def find_similar_wines(df, target_wine_name, ct, columns, top_n=20, final_n=10):
    target_index = find_wine_index(df, target_wine_name)
    distances = calculate_distance_with_flavors(df, target_index, ct, columns)
    
    # 유사도와 데이터프레임 결합 (유클리디안 거리이므로 거리 값이 작은 순서가 유사도 높은 것)
    df['distance'] = distances
    
    # 거리 값이 작은 순으로 정렬
    df_sorted = df.sort_values(by='distance', ascending=True)
    
    # 자기 자신을 제외하고 유사한 와인만 필터링
    df_sorted_filtered = df_sorted[df_sorted['wine_name'] != target_wine_name]
    
    # 상위 top_n 개 와인 선택
    df_top_n = df_sorted_filtered.head(top_n)
    
    # 와인 이름에서 공백 뒤 숫자 제거
    df_top_n['base_name'] = df_top_n['wine_name'].apply(normalize_wine_name)
    
    # 와인 이름별로 가장 유사한 와인만 남기기
    df_top_n_unique = df_top_n.loc[df_top_n.groupby('base_name')['distance'].idxmin()]
    
    # 최종 필터링 및 정렬
    df_final = df_top_n_unique[['wine_name', 'wine_price', 'wine_country', 'point', 'distance']]
    
    return df_final

# 실행 예
target_wine_name = 'Classic Malbec 2019'
columns = ['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']
df_filtered = find_similar_wines(df, target_wine_name, ct, columns)

print("상위 10개의 유사 와인:")
print(df_filtered.head(10))  # 최종 상위 10개 와인 출력


상위 10개의 유사 와인:
                               wine_name  wine_price wine_country     point  \
1587             Auxo Colli di Luni 2020     19736.0        Italy  3.829268   
1038                      Bardolino 2021     14216.0        Italy  3.757886   
1154                     Bell'Assai 2016     58337.0        Italy  3.768411   
1150              Blau & Blau Rosso 2016     36727.0        Italy  3.926766   
338   Colección de los Andes Malbec 2014     21668.0    Argentina  3.322298   
473                   Estate Malbec 2013     20834.0    Argentina  3.525994   
949                          Malbec 2016     40836.0    Argentina  3.069252   
4                      Memoro Rosso N.V.     27085.0        Italy  3.800208   
1131                         Merlot 2017     13709.0        Chile  3.607868   
267          Paso del Sol Carmenère 2004     16251.0        Chile  3.316138   

       distance  
1587  74.503613  
1038  74.359020  
1154  69.023952  
1150  74.049937  
338   67.227132  
473   5

C:\Users\KOSMO\AppData\Local\Temp\ipykernel_25332\1577529157.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_n['base_name'] = df_top_n['wine_name'].apply(normalize_wine_name)


Evenstad Reserve Pinot Noir 2006